In [ ]:
import os
import numpy as np
import jax.numpy as jnp
from jax import random as jran
from diffaux.size_modeling.fit_size_data import (
    assemble_mixed_fit_parameters,
    Samples_MFit,
    generate_sizes,
    DEFAULT_FIT_FUNCTIONS,
    DEFAULT_SIZE_FUNCTIONS,
    DEFAULT_MIXED_FIT_FITTYPES,
)

In [ ]:
fit_pars = assemble_mixed_fit_parameters(Samples_MFit)
print(fit_pars)

In [ ]:
N = 6000
lM_lo = 7.5
lM_hi = 12.0
z_lo = 0.0
z_hi = 3.0
UVJcolor_cut = 1.5
log_Mstar = np.random.uniform(low=lM_lo, high=lM_hi, size=N)
redshift = np.random.uniform(low=z_lo, high=z_hi, size=N)
color_gal = np.random.uniform(low=-0.2, high=2.3, size=N)
mstar = np.power(10, log_Mstar)

In [ ]:
Re, R_med = generate_sizes(
    fit_pars,
    log_Mstar,
    redshift,
    color_gal,
    fit_types=DEFAULT_MIXED_FIT_FITTYPES,
    samples=Samples_MFit,
    size_funcs=DEFAULT_SIZE_FUNCTIONS,
    fit_funcs=DEFAULT_FIT_FUNCTIONS,
    scatter=0.2,
)
color_disk = np.ones(len(log_Mstar)) * 0.5
Re_disk, R_med_disk = generate_sizes(
    fit_pars,
    log_Mstar,
    redshift,
    color_disk,
    fit_types=DEFAULT_MIXED_FIT_FITTYPES,
    samples=Samples_MFit,
    size_funcs=DEFAULT_SIZE_FUNCTIONS,
    fit_funcs=DEFAULT_FIT_FUNCTIONS,
    scatter=0.2,
)
color_bulge = np.ones(len(log_Mstar)) * 2.0
Re_bulge, R_med_bulge = generate_sizes(
    fit_pars,
    log_Mstar,
    redshift,
    color_bulge,
    fit_types=DEFAULT_MIXED_FIT_FITTYPES,
    samples=Samples_MFit,
    size_funcs=DEFAULT_SIZE_FUNCTIONS,
    fit_funcs=DEFAULT_FIT_FUNCTIONS,
    scatter=0.2,
)

In [ ]:
print(np.min(Re_disk), np.max(Re_disk), np.min(R_med_disk), np.max(R_med_disk))
print(np.min(Re_bulge), np.max(Re_bulge), np.min(R_med_bulge), np.max(R_med_bulge))

In [ ]:
mask = color_gal < 1.5
print(R_med[mask])
print(R_med_disk[mask])
print(R_med[~mask])
print(R_med_bulge[~mask])

In [ ]:
# Check with plot
from diffaux.validation.plot_size_fits import plot_generated_sizes
from diffaux.validation.read_size_validation_data import read_size_data, validation_info
from diffaux.validation.read_size_validation_data import get_author_list

authors_re = get_author_list(validation_info["Re_vs_z_data"])
Redata = read_size_data({}, authors_re, info_key="Re_vs_z_data")
authors_rd = get_author_list(validation_info["Re_vs_Mstar_data"])
Rddata = {}
for author in authors_rd:
    samples = validation_info["Re_vs_Mstar_data"][author]["samples"]
    for sample in samples:
        if "All" not in sample:
            Rddata = read_size_data(Rddata, authors_rd, info_key="Re_vs_Mstar_data", sample=sample)
tests = [Redata, Rddata]

In [ ]:
notebook_dir = "/Users/kovacs/cosmology/diff_notebooks"
from diffaux.validation.plot_size_data import PLOT_DRN

for test, ikey in zip(tests, ["Re_vs_z_data", "Re_vs_Mstar_data"]):
    authors = get_author_list(validation_info[ikey])
    plot_generated_sizes(
        Re,
        R_med,
        color_gal,
        log_Mstar,
        redshift,
        Samples_MFit,
        authors,
        test[ikey],
        validation_info[ikey],
        logM_lo=7.5,
        plotdir=os.path.join(notebook_dir, PLOT_DRN),
        pltname="GalaxySizes_vs_Mstar_zbins_MixedFit_{}.png",
    )

## Test diffsky PR

In [ ]:
from diffsky.experimental.size_modeling.disk_bulge_sizes import (
    _disk_median_r50,
    _bulge_median_r50,
    mc_r50_disk_size,
    mc_r50_bulge_size,
)

In [ ]:
# Check median values
r50_med_bulge = _bulge_median_r50(mstar, redshift)
r50_med_disk = _disk_median_r50(mstar, redshift)
print(jnp.min(r50_med_disk), jnp.min(r50_med_bulge), jnp.max(r50_med_disk), jnp.max(r50_med_bulge))
print(r50_med_disk)
print(R_med_disk)
print(r50_med_bulge)
print(R_med_bulge)

mask = color_gal < 1.5
print(r50_med_disk[mask])
print(R_med_disk[mask])
print(r50_med_bulge[~mask])
print(R_med_bulge[~mask])
# assemble test
r50_med_test = np.zeros(len(redshift))
r50_med_test[mask] = r50_med_disk[mask]
r50_med_test[~mask] = r50_med_bulge[~mask]

In [ ]:
ran_key = jran.key(0)
disk_key, ran_key = jran.split(ran_key, 2)
bulge_key, ran_key = jran.split(ran_key, 2)

r50_disk = mc_r50_disk_size(mstar, redshift, disk_key)
r50_bulge = mc_r50_bulge_size(mstar, redshift, bulge_key)
print(jnp.min(r50_disk), jnp.min(r50_bulge), jnp.max(r50_disk), jnp.max(r50_bulge))

In [ ]:
print(Re_disk)
print(r50_disk)
print(Re_bulge)
print(r50_bulge)
# assemble array to make validation plot
r50_test = np.zeros(len(redshift))
r50_test[mask] = r50_disk[mask]
r50_test[~mask] = r50_bulge[~mask]
print(r50_test)
print(Re)

In [ ]:
from importlib.resources import files

SIZE_DATA_DIR = files("diffaux").joinpath("data/GalaxySizes/zhang_2019")
fn = "Zhang_et_al_2019_Size_Morphology.txt"
data = np.genfromtxt(
    os.path.join(SIZE_DATA_DIR, fn),
    skip_header=2,
    dtype=[("type", "U12"), ("alpha", float), ("beta", float), ("gamma", float), ("M0", float)],
    names=True,
)

keys = [
    str(data[k][0]) for k in range(len(data))
]  # if 'B/T' not in k or 'star' not in k or 'quies' not in k]
zhang = dict(zip(keys, [[float(data[k][l]) for l in range(1, len(data[k]))] for k in range(len(data))]))
del zhang["quiescent"]
del zhang["star-forming"]


def zhang_et_al_fit(M, alpha, beta, gamma, M0, h=0.678):
    # correct M to units of Msun/h**2
    M = M / (h * h)
    return gamma * (M / M0) ** alpha * (1 + M / M0) ** (beta - alpha)

In [ ]:
for test, ikey in zip(tests, ["Re_vs_z_data", "Re_vs_Mstar_data"]):
    authors = get_author_list(validation_info[ikey])
    plot_generated_sizes(
        r50_test,
        r50_med_test,
        color_gal,
        log_Mstar,
        redshift,
        Samples_MFit,
        authors,
        test[ikey],
        validation_info[ikey],
        data_fits=zhang,
        sample_labels=["Disk", "Bulge"],
        logM_lo=7.5,
        plotdir=os.path.join(notebook_dir, PLOT_DRN),
        pltname="GalaxySizes_vs_Mstar_zbins_MixedFit_{}_TestDiffsky_zhang.png",
    )